In [2]:
from ibm_watsonx_ai import APIClient
from langchain_ibm import ChatWatsonx

project_id="2adbc31f-5dc4-4e42-b730-1e5e360727fd"
url="https://us-south.ml.cloud.ibm.com"
apikey="0cZb46prORvZpwwPng_oLDJOxt5SmJwRqzw0gAfZKJ5_"

parameters = {
    "frequency_penalty": 0,
    "max_tokens": 100,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

def create_chat_model(model_id):
    return ChatWatsonx(
    model_id=model_id,
    url=url,
    apikey=apikey,
    project_id=project_id,
    params=parameters,
)
model_id = "ibm/granite-3-8b-instruct"
chat_model = create_chat_model(model_id)
chat_model.invoke("hello")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 60, 'total_tokens': 70}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop'}, id='chatcmpl-b354d791e46b1271ac04549852145771', usage_metadata={'input_tokens': 60, 'output_tokens': 10, 'total_tokens': 70})

In [3]:
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from langchain_community.tools.openweathermap import OpenWeatherMapQueryRun
weather_apikey= 'c0a856e7b381cf41550a94999c5d972d'
weather_tool = OpenWeatherMapQueryRun(api_wrapper = OpenWeatherMapAPIWrapper(openweathermap_api_key =weather_apikey ))

In [14]:
from typing import Annotated

from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage

In [12]:
class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)
memory = MemorySaver()

tools = [weather_tool]
llm_with_tools = chat_model.bind_tools(tools)   #Bind tool-like objects to this chat model.

In [13]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
tool_node = ToolNode(tools=[weather_tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")


graph = graph_builder.compile(checkpointer=memory)

In [16]:
config = {"configurable": {"thread_id": "1"}}

messages = "Hi there! My name is Will."
u={"messages": [HumanMessage(content=messages)]}
graph.invoke(u, config = config )

{'messages': [HumanMessage(content='Hi there! My name is Will.', additional_kwargs={}, response_metadata={}, id='1603dd88-ee16-444b-bbc4-0d5a457c810e'),
  AIMessage(content="Hello Will! It's nice to meet you. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 249, 'total_tokens': 267}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop'}, id='chatcmpl-fe70b9d739a45ba8249f6760ab817f4c', usage_metadata={'input_tokens': 249, 'output_tokens': 18, 'total_tokens': 267})]}

In [17]:
messages = "Hi there! what is my name?"
u={"messages": [HumanMessage(content=messages)]}
graph.invoke(u, config = config )

{'messages': [HumanMessage(content='Hi there! My name is Will.', additional_kwargs={}, response_metadata={}, id='1603dd88-ee16-444b-bbc4-0d5a457c810e'),
  AIMessage(content="Hello Will! It's nice to meet you. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 249, 'total_tokens': 267}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop'}, id='chatcmpl-fe70b9d739a45ba8249f6760ab817f4c', usage_metadata={'input_tokens': 249, 'output_tokens': 18, 'total_tokens': 267}),
  HumanMessage(content='Hi there! what is my name?', additional_kwargs={}, response_metadata={}, id='b067190d-73bc-476b-bf3d-a9b07f423169'),
  AIMessage(content='Your name is Will.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 284, 'total_tokens': 290}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop'}, id='chat

In [18]:
config = {"configurable": {"thread_id": "1"}}

messages = "Hi there! what is the temperature in greensboro NC"
u={"messages": [HumanMessage(content=messages)]}
graph.invoke(u, config = config )

{'messages': [HumanMessage(content='Hi there! My name is Will.', additional_kwargs={}, response_metadata={}, id='1603dd88-ee16-444b-bbc4-0d5a457c810e'),
  AIMessage(content="Hello Will! It's nice to meet you. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 249, 'total_tokens': 267}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop'}, id='chatcmpl-fe70b9d739a45ba8249f6760ab817f4c', usage_metadata={'input_tokens': 249, 'output_tokens': 18, 'total_tokens': 267}),
  HumanMessage(content='Hi there! what is my name?', additional_kwargs={}, response_metadata={}, id='b067190d-73bc-476b-bf3d-a9b07f423169'),
  AIMessage(content='Your name is Will.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 284, 'total_tokens': 290}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop'}, id='chat

In [21]:
config = {"configurable": {"thread_id": "1"}}

messages = "Hi there! what is the humidity there"
u={"messages": [HumanMessage(content=messages)]}
graph.invoke(u, config = config )

{'messages': [HumanMessage(content='Hi there! My name is Will.', additional_kwargs={}, response_metadata={}, id='1603dd88-ee16-444b-bbc4-0d5a457c810e'),
  AIMessage(content="Hello Will! It's nice to meet you. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 249, 'total_tokens': 267}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop'}, id='chatcmpl-fe70b9d739a45ba8249f6760ab817f4c', usage_metadata={'input_tokens': 249, 'output_tokens': 18, 'total_tokens': 267}),
  HumanMessage(content='Hi there! what is my name?', additional_kwargs={}, response_metadata={}, id='b067190d-73bc-476b-bf3d-a9b07f423169'),
  AIMessage(content='Your name is Will.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 284, 'total_tokens': 290}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop'}, id='chat

In [20]:
config = {"configurable": {"thread_id": "4"}}

messages = "Hi there! what is the humidity there"
u={"messages": [HumanMessage(content=messages)]}
graph.invoke(u, config = config )

{'messages': [HumanMessage(content='Hi there! what is the humidity there', additional_kwargs={}, response_metadata={}, id='d15fc0cd-7dbb-486b-b13d-7354382c2727'),
  AIMessage(content="I'm sorry for the confusion, but I need a location to provide the current weather information. Could you please specify a location?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 250, 'total_tokens': 277}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop'}, id='chatcmpl-f9b687d88010c1217f1115631aa7e633', usage_metadata={'input_tokens': 250, 'output_tokens': 27, 'total_tokens': 277})]}

# Human-in-the-loop

In [62]:
from langchain_core.tools import tool
from langgraph.types import Command, interrupt

In [63]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [64]:
@tool
def human_assistance(query: str) -> str:
    """Request assistance from a human."""
    human_response = interrupt({"query": query})
    return human_response["data"]

tools = [weather_tool]
llm_with_tools = chat_model.bind_tools(tools)

In [65]:
def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    # Because we will be interrupting during tool execution,
    # we disable parallel tool calling to avoid repeating any
    # tool invocations when we resume.
    assert len(message.tool_calls) <= 1
    return {"messages": [message]}

In [66]:
graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)

In [67]:
messages = "Hi there! I need help finding temperatur in my city?"
u={"messages": [HumanMessage(content=messages)]}
graph.invoke(u, config = config )

{'messages': [HumanMessage(content='Hi there! I need help finding temperatur in my city?', additional_kwargs={}, response_metadata={}, id='53685e6a-e186-4bb9-a8c9-36293822e197'),
  AIMessage(content='Of course, I can help with that. Could you please tell me the name of your city?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 255, 'total_tokens': 276}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop'}, id='chatcmpl-5284d2d526cf0b05d263a244e78897f0', usage_metadata={'input_tokens': 255, 'output_tokens': 21, 'total_tokens': 276})]}

In [68]:
messages = "yes. i live in Paris"
u={"messages": [HumanMessage(content=messages)]}
graph.invoke(u, config = config )

{'messages': [HumanMessage(content='Hi there! I need help finding temperatur in my city?', additional_kwargs={}, response_metadata={}, id='53685e6a-e186-4bb9-a8c9-36293822e197'),
  AIMessage(content='Of course, I can help with that. Could you please tell me the name of your city?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 255, 'total_tokens': 276}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop'}, id='chatcmpl-5284d2d526cf0b05d263a244e78897f0', usage_metadata={'input_tokens': 255, 'output_tokens': 21, 'total_tokens': 276}),
  HumanMessage(content='yes. i live in Paris', additional_kwargs={}, response_metadata={}, id='b63deafc-d20b-422c-9232-de0aab8fe296'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-48f2c048a9084d50a4a51eb46ff00422', 'type': 'function', 'function': {'name': 'open_weather_map', 'arguments': '{"location": "Paris,FR"}'}}]}, response_me

# Customizing State

In [ ]:
complete_query = "Your task is to provide city name based on the user query. \
         If the use enters the state, provide also the state. Return city and state in a comma separated format. \
        Following is the user query: " + input_1

In [83]:
prompt = '''your task is to generate a line of code that can be used to modify the InputTables.
    Use the examples below:
    "double the transporation cost": "so.InputTables['transportation']['cost']= 2* so.InputTables['transportation']['cost']"   
    "Increase the transportation cost by 20 percent: "so.InputTables['transportation']['cost']= so.InputTables['transportation']['cost'] * (1.2)" 
     Following is the user query: '''

u= chat_model.invoke(prompt+"triple the transportation cost")

In [84]:
u.content

"so.InputTables['transportation']['cost'] = 3 * so.InputTables['transportation']['cost']"